In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# train data
train_data = joblib.load('./x_train.lz4')
data_label = pd.read_csv('../../preprocess_data/train_y_33465.csv',usecols=['label'])
x_train = train_data.fillna(-1).values
y_train = data_label.values.ravel()
# valid data
valid_data = joblib.load('./x_valid.lz4')
x_valid = valid_data.fillna(-1).values

In [ ]:
# train
import lightgbm as lgb
parameters = {
    'boost':'gbdt',
    'num_leaves':150, 
    'scale_pos_weight':float(len(y_train)-np.sum(y_train))/float(np.sum(y_train)),
    'max_depth':-1,
    'learning_rate':.05,
    'max_bin':250,
    'min_data_in_leaf' : 60,
    'objective':'binary',
    'metric':'auc',
    'num_threads':24
}
lgb_train = lgb.Dataset(x_train, y_train)
lgb_model = lgb.train(parameters,lgb_train,num_boost_round=200,verbose_eval=True)

In [ ]:
joblib.dump(lgb_model,'./model/lgb_model')

In [ ]:
# predict
valid_id = pd.read_csv('../../preprocess_data_new/valid_date.csv',usecols=['id']).values.ravel()
model = joblib.load('./model/lgb_model')
def predict_prob(model,x_test,test_id):
    pred = pd.DataFrame()
    pred['id'] = test_id
    pred['prob'] = model.predict(x_test)
    return pred

In [ ]:
x_train.shape,x_valid.shape

In [ ]:
pred = predict_prob(model,x_valid,valid_id)

In [ ]:
pred.to_csv('./preds/lgb_pred.txt',index=False)

In [1]:
# AUC：0.819